In [2]:
# back calculation
#####################



import time  # to simulate a real time data, time loop
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import yahoo_fin.stock_info as si
import numpy as np  # np mean, np random
import pandas as pd  # read csv, df manipulation
import plotly.express as px  # interactive charts
import streamlit as st  # 🎈 data web app development
import datetime
from datetime import date
import plotly.express as px




##########################


def calculate_recom(stock_id, current_date):

    #current_date = datetime.datetime.now()
    # 5 weeks
    pred_day = current_date - datetime.timedelta(days=35)

    sdate = current_date.strftime('%Y-%m-%d')
    pred_day = pred_day.strftime('%Y-%m-%d')

    df1 = yf.download(
                    tickers = stock_id,
                    start=pred_day,
                    #end=date.today().replace(day=2),
                    end = sdate,
                    interval = "1d",
                    threads=True,
                    group_by = 'ticker', 
                    progress=False
                )
    
    #print(df1.tail(3))
    #print('############')
    #print(current_date)
    #print('!!!!!!!!!!!!!!!!############!!!!!!!!!!!!!!!!')



    
    #if (df1.index[-1] == current_date - datetime.timedelta(days=1)) | (df1.index[-1] == current_date - datetime.timedelta(days=3)):
    #    pass
    #else: return (sdate, np.NaN )


    df1 = df1[['Adj Close', 'Volume']]
    df1.columns = ['Price', 'Vol']

    df1 = df1[df1['Vol'] != 0]



    df1['P5'] = df1['Price'].shift(5)
    df1['V5'] = df1['Vol'].shift(5)

    df1['PC'] = df1['Price'] - df1['P5']
    df1['VC'] = df1['Vol'] - df1['V5']

    #df1['N50'] = nifty['Adj Close']

    df1 = df1.dropna()

    #st.dataframe(df1)

    l1 = []

    l1 = list(df1.resample('7D')['VC'].mean().tail(5))
    #st.write(l1)
    recom = round((sum([i for i in l1 if i >= 0 ]) / sum([abs(number) for number in l1]))*100)
    
    return (sdate, recom)

def get_recoms(stock_id):
    recoms = []
    sdate = datetime.datetime(2016, 1, 1)
    edate = datetime.datetime(2024, 1, 31)
    from datetime import date, timedelta

    days = [sdate+timedelta(days=x) for x in range((edate-sdate).days)]


    for d in days:
        if d.weekday() < 5:
            recoms.append(calculate_recom(stock_id, d))

    return recoms



nse_df = pd.read_excel('nse_data.xlsx')
nse_df = nse_df.tail(2)
nse_df['Symbol'] = [str(s) + '.NS' for s in nse_df['Symbol']]


for stock_id in list(set(nse_df['Symbol'])):
    try:
        df = pd.DataFrame(get_recoms(stock_id), columns=['date', 'recom'])
        df.to_csv('stocks\\' + str(stock_id.split('.')[0]) +'.csv', index= False)
    except:
        continue

ValueError: Length of values (1) does not match length of index (50)